In [1]:
import torch
import numpy as np
import datasets
import pickle
import pathlib
import os
from importlib import reload
import sys
import pandas as pd
from IPython.display import display, HTML
import json
import sklearn
from sklearn.metrics import accuracy_score
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import logging

from tqdm.auto import tqdm
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers.trainer_pt_utils import LengthGroupedSampler
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding
)

# For Imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# print("sys.path:", sys.path)
plt.style.use("ggplot")
plt.style.context("ggplot")

2023-01-21 06:43:11.343907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 06:43:11.520049: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-21 06:43:12.226133: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/modaresi/.conda/envs/globenc-venv/lib/
2023-01-21 06:43:12.226245: W tensorflow/

# Configs

In [11]:
MODELS_DIR = "/home/modaresi/projects/globenc_analysis/outputs/models"
OUTPUT_DIR = "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3"
IG_OUTPUT_DIR = "/home/modaresi/projects/globenc_analysis/outputs/integrated_saliencies"
IXG_OUTPUT_DIR = "/home/modaresi/projects/globenc_analysis/outputs/saliencies"
ALTI_OUTPUT_DIR = "/home/modaresi/projects/globenc_analysis/outputs/ALTI"

DIR_MODEL_DATASET_SET = [
    (OUTPUT_DIR, f"{MODELS_DIR}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
#     (OUTPUT_DIR, f"{MODELS_DIR}/output_hatexplain_bert-base-uncased_0001_SEED0042/checkpoint-2405", "hatexplain", "validation"),
#     (OUTPUT_DIR, f"{MODELS_DIR}/output_qnli_bert-base-uncased_0001_SEED0042/checkpoint-16370", "qnli", "validation"),
#     (OUTPUT_DIR, f"{MODELS_DIR}/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360", "mnli", "validation_matched"),
    
#     (f"{MODELS_DIR}/output_cola_bert-base-uncased_0001_SEED0042/checkpoint-1340", "cola", "validation"),
#     (f"{MODELS_DIR}/output_mrpc_bert-base-uncased_0001_SEED0042/checkpoint-575", "mrpc", "validation"),
#     (f"{MODELS_DIR}/output_sst2_bert-large-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
### IG ###
    (IG_OUTPUT_DIR, f"{MODELS_DIR}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
#     (IG_OUTPUT_DIR, f"{MODELS_DIR}/output_hatexplain_bert-base-uncased_0001_SEED0042/checkpoint-2405", "hatexplain", "validation"),
#     (IG_OUTPUT_DIR, f"{MODELS_DIR}/output_qnli_bert-base-uncased_0001_SEED0042/checkpoint-16370", "qnli", "validation"),
#     (IG_OUTPUT_DIR, f"{MODELS_DIR}/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360", "mnli", "validation_matched"),
### IXG ###
    (IXG_OUTPUT_DIR, f"{MODELS_DIR}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
#     (IXG_OUTPUT_DIR, f"{MODELS_DIR}/output_hatexplain_bert-base-uncased_0001_SEED0042/checkpoint-2405", "hatexplain", "validation"),
#     (IXG_OUTPUT_DIR, f"{MODELS_DIR}/output_qnli_bert-base-uncased_0001_SEED0042/checkpoint-16370", "qnli", "validation"),
#     (IXG_OUTPUT_DIR, f"{MODELS_DIR}/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360", "mnli", "validation_matched"),
### ALTI ###
    (ALTI_OUTPUT_DIR, f"{MODELS_DIR}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
#     (ALTI_OUTPUT_DIR, f"{MODELS_DIR}/output_hatexplain_bert-base-uncased_0001_SEED0042/checkpoint-2405", "hatexplain", "validation"),
#     (ALTI_OUTPUT_DIR, f"{MODELS_DIR}/output_qnli_bert-base-uncased_0001_SEED0042/checkpoint-16370", "qnli", "validation"),
#     (ALTI_OUTPUT_DIR, f"{MODELS_DIR}/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360", "mnli", "validation_matched"),
### RoBERTa ###
#     (OUTPUT_DIR, f"WillHeld/roberta-base-sst2", "sst2", "validation"),
#     (ALTI_OUTPUT_DIR, f"WillHeld/roberta-base-sst2", "sst2", "validation"),
#     (OUTPUT_DIR, f"WillHeld/roberta-base-mnli", "mnli", "validation_matched"),
#     (ALTI_OUTPUT_DIR, f"WillHeld/roberta-base-mnli", "mnli", "validation_matched"),
#     (IG_OUTPUT_DIR, f"WillHeld/roberta-base-sst2", "sst2", "validation"),
#     (IXG_OUTPUT_DIR, f"WillHeld/roberta-base-sst2", "sst2", "validation"),
#     (IG_OUTPUT_DIR, f"WillHeld/roberta-base-mnli", "mnli", "validation_matched"),
#     (IXG_OUTPUT_DIR, f"WillHeld/roberta-base-mnli", "mnli", "validation_matched"),
]

GLOBENC_CONFIG_NAMES = [
    'Decomposition AbsDot Bias',
#     '+Decomposition AbsDot Bias',
    'GlobEnc', 
#     'GlobEnc No Bias FFN', 
#     'Decomposition No Bias', 
#     'Decomposition Equal Bias', 'Decomposition Norm Bias', 'Decomposition AbsSim Bias', 
#     'Decomposition AbsDot Bias No FFN'
#     'GlobEnc AbsDot Bias', 'GlobEnc AbsSim Bias', 'GlobEnc Equal Bias', 'GlobEnc Norm Bias', 
#     'GlobEnc AbsDot Bias FFN', 'GlobEnc Equal Bias FFN', 'GlobEnc AbsSim Bias FFN', 'GlobEnc Norm Bias FFN', 
#     'Decomposition No Bias No FFN',
    
    ### IG ###
    "IG_ALL_ZERO_NORM_prediction_based",
#     "IG_ALL_PAD_SUM_prediction_based",
#     "IG_ALL_PAD_NORM_prediction_based",
    "IG_ALL_ZERO_SUM_prediction_based",
#     "IG_ALL_PAD_SUM_label_based",
#     "IG_ALL_PAD_NORM_label_based",
#     "IG_ALL_ZERO_SUM_label_based",
#     "IG_ALL_ZERO_NORM_label_based",

    ### IXG ###
    "IXG_NORM_prediction_based",
    "IXG_SUM_prediction_based",
    
    ### ALTI ###
    "ALTI"
]

In [12]:
def print_importance(importance, tokenized_text, discrete=False, prefix="", no_cls_sep=False):
    """
    importance: (sent_len)
    """
    if no_cls_sep:
        importance = importance[1:-1]
        tokenized_text = tokenized_text[1:-1]
    importance = importance / np.abs(importance).max() / 1.5  # Normalize
    if discrete:
        importance = np.argsort(np.argsort(importance)) / len(importance) / 1.6
    
    html = "<pre>"+prefix
#     plt.plot(importance)
    for i in range(len(tokenized_text)):
        if importance[i] >= 0:
            rgba = matplotlib.cm.get_cmap('Greens')(importance[i])   # Wistia
        else:
            rgba = matplotlib.cm.get_cmap('Reds')(np.abs(importance[i]))   # Wistia
        text_color = "color: rgba(255, 255, 255, 1.0); " if np.abs(importance[i]) > 0.9 else ""
        color = f"background-color: rgba({rgba[0]*255}, {rgba[1]*255}, {rgba[2]*255}, {rgba[3]}); " + text_color
        html += (f"<span style='"
                 f"{color}"
                 f"border-radius: 5px; padding: 3px;"
#                  f"background-color: rgba(200, {cls_attention[i]*255}, 10, 1.0); "
#                  f"font-size: {int(cls_attention[i]*18 + 1)}px; "
#                  f"font-weight: {int(cls_attention[i]*900)};"
                 f"font-weight: {int(800)};"
                 "'>")
        html += tokenized_text[i].replace('<', "[").replace(">", "]")
        html += "</span> "
    display(HTML(html))
#     print(html)
    return html

def make_result_filename(task_name, set_of_data, model_checkpoint, cfg_name):
    file_name = f"[{task_name}]_[{set_of_data}]_[{'-'.join(model_checkpoint.split('/')[-2:])}]_[{cfg_name}]"
    return file_name

# Preview

In [14]:
# Read Globencs
# globencs = dict()

def print_preview(idx=0, discrete=False):
    NO_CLS_SEP = True
    for dir_model_dataset_set in tqdm(DIR_MODEL_DATASET_SET, desc="Models_Dataset_Sets"):
        dir, model_checkpoint, task_name, set_of_data = dir_model_dataset_set
        for cfg_name in tqdm(GLOBENC_CONFIG_NAMES, desc="Configs"):
            file_name = make_result_filename(task_name, set_of_data, model_checkpoint, cfg_name)
#             print(f"{dir}/{file_name}.pkl")
            try:
                df = pd.read_pickle(f"{dir}/{file_name}.pkl")
            except Exception as e:
    #             logging.warn(e)
                continue

            print(file_name)
    #         print("LOGITS:", df.head(1))
            for col in ["importance_last_layer_aggregated", "importance_last_layer_classifier"]:
                if col in df and df[col][idx] is not None:
                    if "aggregated" in col:
                        sentence_importance = df[col].iloc[idx][0, :]
#                         print(sentence_importance)
                    if "classifier" in col:
                        for label in range(df[col].iloc[idx].shape[-1]):
                            sentence_importance = df[col].iloc[idx][:, label]
#                             print(sentence_importance)
                            print_importance(
                                sentence_importance,
                                df["tokens"].iloc[idx], 
                                prefix=f"{col.split('_')[-1]}L{label}|L={df['label'].iloc[idx]}:".ljust(20),
                                no_cls_sep=NO_CLS_SEP,
                                discrete=False
                            )
                        break
                        sentence_importance = df[col].iloc[idx][:, df["label"].iloc[idx]]
                    if "pooler" in col:
                        sentence_importance = df[col].iloc[idx]
                    print_importance(
                        sentence_importance,
                        df["tokens"].iloc[idx], 
                        prefix=f"{col.split('_')[-1]}:".ljust(20),
                        no_cls_sep=NO_CLS_SEP,
                        discrete=discrete
                    )
            print("------------------------------------")
    return df

df = print_preview(idx=424)  # 2475
# df.head(1)

Models_Dataset_Sets:   0%|          | 0/4 [00:00<?, ?it/s]

Configs:   0%|          | 0/7 [00:00<?, ?it/s]

[sst2]_[validation]_[output_sst2_bert-base-uncased_0001_SEED0042-checkpoint-10525]_[Decomposition AbsDot Bias]


------------------------------------
[sst2]_[validation]_[output_sst2_bert-base-uncased_0001_SEED0042-checkpoint-10525]_[GlobEnc]


------------------------------------


...

------------------------------------
[sst2]_[validation]_[output_sst2_bert-base-uncased_0001_SEED0042-checkpoint-10525]_[IXG_SUM_prediction_based]


------------------------------------


Configs:   0%|          | 0/7 [00:00<?, ?it/s]

[sst2]_[validation]_[output_sst2_bert-base-uncased_0001_SEED0042-checkpoint-10525]_[ALTI]


------------------------------------


In [ ]:
df["tokens"].iloc[0]

In [ ]:
ds = datasets.load_dataset("glue", DIR_MODEL_DATASET_SET[0][2], download_config=datasets.DownloadConfig(local_files_only=True))
# print(ds)
# print(ds[DIR_MODEL_DATASET_SET[0][3]]["idx"][:10])
# print("Check above to be ordered!")
# df[df["tokens"].str.len() < 30].index

# MASKING

In [ ]:
class CheckpointToLogit:
    def __init__(
        self,
        model_checkpoint: str,
        task_name: str,
        set_of_data: str,

        token_importance,  # (N, seq_len)
        masking_ratio: float,
        masking_strategy = "min",  # max | min
        mask_token = "mask",  # mask | pad
        
        save_cls: bool = True,
    ) -> pd.DataFrame:
        self.model_checkpoint = model_checkpoint
        self.task_name = task_name
        self.set_of_data = set_of_data
        self.save_cls = save_cls
        
        self.token_importance = token_importance
        self.masking_ratio = masking_ratio
        self.masking_strategy = masking_strategy
        self.mask_token = mask_token
    
    def retrieve(self) -> pd.DataFrame:
        DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
        # DATASET
        GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
        BATCH_SIZE = 8
        MAX_LENGTH = 128
        def aggregate_hatexplain(example):
            def mode(lst):
                return max(set(lst), key=lst.count)
            example["label"] = mode(example["annotators"]["label"])
            example["text"] = " ".join(example["post_tokens"])
            return example
        actual_task = "mnli" if self.task_name == "mnli-mm" else self.task_name
        if self.task_name in GLUE_TASKS:
            dataset = datasets.load_dataset("glue", actual_task, download_config=datasets.DownloadConfig(local_files_only=True))
        elif self.task_name == "hatexplain":
            dataset = datasets.load_dataset("hatexplain", download_config=datasets.DownloadConfig(local_files_only=True)).map(aggregate_hatexplain)
            for part in ["train", "validation", "test"]:
                dataset[part] = dataset[part].add_column("idx", [i for i in range(len(dataset[part]))])
#         metric = datasets.load_metric('glue', actual_task)
        task_to_keys = {
            "cola": ("sentence", None),
            "mnli": ("premise", "hypothesis"),
            "mnli-mm": ("premise", "hypothesis"),
            "mrpc": ("sentence1", "sentence2"),
            "qnli": ("question", "sentence"),
            "qqp": ("question1", "question2"),
            "rte": ("sentence1", "sentence2"),
            "sst2": ("sentence", None),
            "stsb": ("sentence1", "sentence2"),
            "wnli": ("sentence1", "sentence2"),
            "hatexplain": ("text", None),
        }
        SENTENCE1_KEY, SENTENCE2_KEY = task_to_keys[self.task_name]
        def preprocess_function_wrapped(tokenizer):
            def preprocess_function(examples):
                # Tokenize the texts
                args = (
                    (examples[SENTENCE1_KEY],) if SENTENCE2_KEY is None else (examples[SENTENCE1_KEY], examples[SENTENCE2_KEY])
                )
                result = tokenizer(*args, padding=False, max_length=MAX_LENGTH, truncation=True)
                return result
            return preprocess_function

        def token_id_to_tokens_mapper(tokenizer, sample):
            length = len(sample["input_ids"])
            return tokenizer.convert_ids_to_tokens(sample["input_ids"])[:length], length
        
        def mask(example):
            length = np.sum(example["attention_mask"])
            def top_k_masking():
                rank = self.token_importance[example["idx"]][:length].argsort()

                # Exclude CLS and SEPs
                cls_id = tokenizer.cls_token_id
                sep_id = tokenizer.sep_token_id
#                 print("CLS, SEP:", cls_id, sep_id)
                rank = rank[~np.in1d(rank, np.argwhere(((np.array(example["input_ids"])==cls_id) | (np.array(example["input_ids"])==sep_id)).flatten()))]
                
                mask_count = int(np.floor(len(rank) * self.masking_ratio))
                masks = []
                if self.masking_strategy == "max":
                    masks = rank[-mask_count:]
                elif self.masking_strategy == "min":
                    masks = rank[:mask_count]
                else:
                    raise Exception("Unknow masking_strategy. Options are 'max' or 'min'")
                if mask_count == 0:
                    masks = []
                return masks
            
            def top_p_masking():
                p = self.token_importance[example["idx"]][:length]
                p = p / np.sum(p)
                p_argsort = p.argsort()
                if self.masking_strategy == "max":
                    p_argsort = np.flip(p_argsort)
                masks = []
                cumulative_p = 0.0
                for i in range(length):
                    if example["input_ids"][p_argsort[i]] >= 103:  # Exclude CLS and SEPs
                        cumulative_p += p[p_argsort[i]]
                        if cumulative_p <= self.masking_ratio:
                            masks.append(p_argsort[i])
                return masks
            
            replacement_token_map = {
                "mask": tokenizer.mask_token_id,
                "pad": tokenizer.pad_token_id
            }
            replacement_token = replacement_token_map[self.mask_token]
            masks = top_k_masking()
            example["input_ids"] = [replacement_token if j in masks else example["input_ids"][j] for j in range(length)]
            return example
            
        # RUN
        model = AutoModelForSequenceClassification.from_pretrained(self.model_checkpoint)
        model.eval()
        tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint, use_fast=True, max_length=128)

        sel_dataset = dataset[self.set_of_data].map(preprocess_function_wrapped(tokenizer), batched=True, batch_size=1024)
        ### MASK ###
        sel_dataset = sel_dataset.map(mask)
        ### MASK ###
        dataset_size = len(sel_dataset)
        steps = int(np.ceil(dataset_size / BATCH_SIZE))

        globencs = {
            "tokens": [],
            "logits": [],
            "labels": [],
        }
        lengths = []
        
        for i in tqdm(range(dataset_size), desc="Tokenize Masked"):
            tokens, length = token_id_to_tokens_mapper(tokenizer, sel_dataset[i])
            globencs["labels"].append(sel_dataset[i]["label"])
            globencs["tokens"].append(tokens)
            lengths.append(length)

        generator = torch.Generator()
        generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))

        sampler = LengthGroupedSampler(
            BATCH_SIZE,
            lengths=lengths,
            model_input_name=tokenizer.model_input_names[0],
            generator=generator,
        )

        collator = DataCollatorWithPadding(
            tokenizer=tokenizer
        )

        sel_dataset = sel_dataset.add_column("length", lengths)
#         sel_dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "length", "idx"])
        cols = ["input_ids", "attention_mask", "length", "idx"]
        cols = cols + ["token_type_ids"] if not "roberta" in self.model_checkpoint else cols
        sel_dataset.set_format(type="torch", columns=cols)

        dataloader = DataLoader(
            sel_dataset,
            batch_size=BATCH_SIZE,
            sampler=sampler,
            collate_fn=collator
        )

        model.to(DEVICE)

        it = iter(dataloader)
        idxes = []
        shuffled_globencs, shuffled_cls, shuffled_globencs_all_layers, shuffled_logits = [], [], [], []
        with torch.no_grad():
            for i in tqdm(range(steps), desc="Forward"):
                batch = next(it)
#                 input_batch = {k: batch[k].to(DEVICE) for k in ["input_ids", "token_type_ids", "attention_mask"]}
                input_batch = {k: batch[k].to(DEVICE) for k in batch.keys() - ['idx', 'length']}
                logits, hidden_states = model(
                    **input_batch, 
                    output_attentions=False, 
                    return_dict=False, 
                    output_hidden_states=True, 
                )
                batch_lengths = batch["length"].numpy()
                idxes.extend(batch['idx'].tolist())
                shuffled_logits.extend(logits.cpu().numpy())
                if self.save_cls:
                    cls_repr = hidden_states[-1][:, 0, :].cpu().numpy()  # [13, batch, len, 768]
                    shuffled_cls.extend(cls_repr)

        inverse_idxes = np.argsort(idxes)
        globencs["logits"] = [shuffled_logits[inverse_idxes[i]] for i in range(dataset_size)]
        globencs["label"] = sel_dataset["label"]
        if self.save_cls:
            globencs["cls"] = [shuffled_cls[inverse_idxes[i]] for i in range(dataset_size)]
        return pd.DataFrame(globencs)

In [ ]:
class Results:
    def __init__(self, strategy="max"):
        # data
        self.data = dict()
        # indices
        self.indices = {
            "masking_ratios": set(), 
            "model_checkpoint": set(), 
            "task_name": set(), 
            "set_of_data": set(), 
            "globenc_config_name": set(),
            "output_type": set(),
        }
        # Saving path
        self.save_path = f"results-{strategy}.pkl"
        self.load()
    
    def _make_index_name(self, index_to_value: dict):
        return "_".join([f"[{k}:{v}]" for k, v in index_to_value.items()])
    
    def add(self, values, index_to_value: dict):
        self.data[self._make_index_name(index_to_value)] = values
        for key in self.indices.keys():
            self.indices[key].add(index_to_value[key])
#         self.save()
        
    def get(self, index_to_value: dict):
        result = self.data[self._make_index_name(index_to_value)]
        return result
    
    def summary(self):
        return self.indices
    
    def save(self):
        with open(self.save_path, 'wb') as f:
            print(f"Saving {self.save_path} ...")
            pickle.dump(self.__dict__, f, pickle.HIGHEST_PROTOCOL)
            print(f"Saved {self.save_path}")
    
    def load(self):
        try:
            with open(self.save_path, 'rb') as f:
                print(f"Loading {self.save_path} ...")
                self.__dict__ = pickle.load(f)
                print(f"Loaded {self.save_path}")
        except:
            print(f"Could not load past results from {self.save_path}!")


MASKING_STRATEGY = "max"  # max | min
results = Results(MASKING_STRATEGY)

In [ ]:
MASKING_RATIOS = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # Must include 0.0 for AOPC  [0.0, 0.2, 0.5, 0.7, 0.8, 0.9] [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for masking_ratio in tqdm(MASKING_RATIOS, desc="Ratios"):
    for dir_model_dataset_set in tqdm(DIR_MODEL_DATASET_SET, desc="Models_Dataset_Sets"):
        dir, model_checkpoint, task_name, set_of_data = dir_model_dataset_set
        for gcfg in tqdm(GLOBENC_CONFIG_NAMES, desc="Configs"):
            file_name = f"[{task_name}]_[{set_of_data}]_[{'-'.join(model_checkpoint.split('/')[-2:])}]_[{gcfg}]"
            print("Reading", file_name)
            try:
                df = pd.read_pickle(f"{dir}/{file_name}.pkl")
            except Exception as e:
                logging.warn(e)
                continue
            for col in tqdm(["importance_last_layer_aggregated", "importance_last_layer_classifier"]):
                if col not in df or df[col][0] is None:  # For globenc/IG which has no classificier
                    continue
                if "aggregated" in col:
                    token_importance = [v[0, :] for v in df[col].values]
                if "classifier" in col:
#                     token_importance = [v[:, df["label"].iloc[i]] for i, v in enumerate(df[col].values)]
                    token_importance = [row[col][:, row["logits"].argmax()] for i, row in df.iterrows()]
                idx = 0
                print_importance(
                    token_importance[idx], df["tokens"].iloc[idx], prefix=f"{gcfg}_{col.split('_')[-1]}:".ljust(13), 
                    no_cls_sep=False,
                    discrete=False
                )
                ctl = CheckpointToLogit(
                    model_checkpoint=model_checkpoint,
                    task_name=task_name,
                    set_of_data=set_of_data,
                    save_cls=False,
                    token_importance=token_importance,
                    masking_ratio=masking_ratio,
                    masking_strategy = MASKING_STRATEGY,  # max | min
                    mask_token = "mask",  # mask | pad
                )
                mask_df = ctl.retrieve()
                idx_values = {
                    "masking_ratios": masking_ratio, 
                    "model_checkpoint": model_checkpoint, 
                    "task_name": task_name, 
                    "set_of_data": set_of_data, 
                    "globenc_config_name": gcfg,
                    "output_type": col.split("_")[-1]
                }
                results.add(mask_df, idx_values)
                print(idx_values)
                display(mask_df.head(1))
results.save()

In [ ]:
def compute_aopc(results: Results, index_to_value: dict):
    result = results.get(index_to_value)
    logits = scipy.special.softmax(np.array(result["logits"].tolist()), axis=-1)
    
    base_index_to_value = index_to_value.copy()
    base_index_to_value["masking_ratios"] = 0.0
    base_result = results.get(base_index_to_value)
    base_logits = scipy.special.softmax(np.array(base_result["logits"].tolist()), axis=-1)
    
    p_hat_idx = np.expand_dims(base_logits.argmax(axis=-1), -1)  # (N, 1)
    masked_p = np.take_along_axis(logits, p_hat_idx, axis=-1)  # Select p of the predicted label
    base_p = np.take_along_axis(base_logits, p_hat_idx, axis=-1)  # Select p of the predicted label
    aopc = np.mean(base_p - masked_p)
#     aopc = np.mean(np.abs(base_p - masked_p))
    return aopc

def compute_accuracy(results: Results, index_to_value: dict):
    result = results.get(index_to_value)
    preds = np.argmax(np.array(result["logits"].tolist()), axis=-1)
    labels = result["label"].values
    return accuracy_score(y_true=labels, y_pred=preds)

def compute_lodds(results: Results, index_to_value: dict):
    result = results.get(index_to_value)
    logits = scipy.special.softmax(np.array(result["logits"].tolist()), axis=-1)
    
    base_index_to_value = index_to_value.copy()
    base_index_to_value["masking_ratios"] = 0.0
    base_result = results.get(base_index_to_value)
    base_logits = scipy.special.softmax(np.array(base_result["logits"].tolist()), axis=-1)
    
    p_hat_idx = np.expand_dims(base_logits.argmax(axis=-1), -1)  # (N, 1)
    masked_p = np.take_along_axis(logits, p_hat_idx, axis=-1)  # Select p of predicted label
    base_p = np.take_along_axis(base_logits, p_hat_idx, axis=-1)  # Select p of predicted label
    lodds = np.mean(np.log(masked_p / base_p))
    return lodds

def compute_mae(results: Results, index_to_value: dict):
    result = results.get(index_to_value)
    logits = scipy.special.softmax(np.array(result["logits"].tolist()), axis=-1)
    base_index_to_value = index_to_value.copy()
    base_index_to_value["masking_ratios"] = 0.0
    base_result = results.get(base_index_to_value)
    base_logits = scipy.special.softmax(np.array(base_result["logits"].tolist()), axis=-1)
    
    p_hat_idx = np.expand_dims(base_logits.argmax(axis=-1), -1)  # (N, 1)
    masked_p = np.take_along_axis(logits, p_hat_idx, axis=-1)  # Select p of predicted label
    base_p = np.take_along_axis(base_logits, p_hat_idx, axis=-1)  # Select p of predicted label
    mae = np.mean(np.abs(masked_p - base_p))
    return mae

metric_function = {
    "Accuracy": compute_accuracy,
    "AOPC": compute_aopc,
    "LOdds": compute_lodds,
    "AOPC+": compute_mae,
}
METRIC = "Accuracy"
filter_dataset = "sst2"  # None sst2 mnli qnli hatexplain
filter_exclude_configs = [] # ["Decomposition Equal Bias", "Decomposition Norm Bias", "Decomposition ReLU"]

plt.figure(figsize=(11, 8))
LEGEND_MAP = {
    "aggregated Decomposition": "Decomposition",
#     "aggregated GlobEnc": "GlobEnc",
#     "aggregated IG_ALL_PAD_SUM_prediction_based": "Integrated Gradient",
    "aggregated IXG_NORM_prediction_based": "Gradient×Input",
    "classifier Decomposition": "Decomposition + Classification Head",
}
markers = ['-D', '-o', '-^', '-*', '-s', '-+', '-x', '-.', '-X']
cmap = plt.get_cmap("tab10")
c = 0
for output_type in sorted(results.indices['output_type']):
    for globenc_config_name in tqdm(sorted(results.indices['globenc_config_name'])):
        for model_checkpoint in results.indices['model_checkpoint']:
            for task_name in results.indices['task_name']:
                for set_of_data in results.indices['set_of_data']:
                    if filter_dataset and filter_dataset not in f"{task_name}_{globenc_config_name}_{output_type}":
                        continue
#                     if any([f in globenc_config_name for f in filter_exclude_configs]):
#                         continue
#                     if output_type == "aggregated":
#                         continue
                    plot_y, plot_x = [], []
                    try:
                        for masking_ratio in sorted(results.indices['masking_ratios']):
                            if masking_ratio == 1.0:
                                continue
                            index_to_value = {
                                "masking_ratios": masking_ratio, 
                                "model_checkpoint": model_checkpoint, 
                                "task_name": task_name, 
                                "set_of_data": set_of_data, 
                                "globenc_config_name": globenc_config_name,
                                "output_type": output_type,
                            }
                            metric_result = metric_function[METRIC](results, index_to_value)
                            plot_y.append(metric_result)
                            plot_x.append(masking_ratio)
                        label = f"{task_name} {output_type} {globenc_config_name}"
                        label = LEGEND_MAP.get(label, label)
#                         print(label)
                        if ("Decomp" in label or "ALTI" in label or "GlobEnc" in label) and "roberta" in model_checkpoint:
#                             print(label)
                            plt.plot(plot_x, plot_y, markers[c], color=cmap(c), label=label)
                            c += 1
                        print(f"{output_type} {globenc_config_name}".ljust(40), f"--> mean={np.mean(plot_y[1:])}, {len(plot_y[1:])}")
                    except Exception as e:
#                         logginginging.warning(e)
                        pass
plt.legend(facecolor='white', framealpha=0.8)
plt.title(f"{filter_dataset} Mask {MASKING_STRATEGY} top_k")
plt.ylabel(f"{METRIC}")
plt.xlabel("k")
plt.show()
print(model_checkpoint)

# Extreme Examples

In [ ]:
from scipy import stats

output_type = 'aggregated'  # 'classifier'

# i1 = {
#     'masking_ratios': 0.9, 
#     'model_checkpoint': '/home/modaresi/projects/globenc_analysis/outputs/models/output_qnli_bert-base-uncased_0001_SEED0042/checkpoint-16370', 
#     'task_name': 'qnli', 
#     'set_of_data': 'validation', 
#     'globenc_config_name': 'Decomposition No Bias', 
#     'output_type': output_type
# }
# i2 = {
#     'masking_ratios': 0.9, 
#     'model_checkpoint': '/home/modaresi/projects/globenc_analysis/outputs/models/output_qnli_bert-base-uncased_0001_SEED0042/checkpoint-16370', 
#     'task_name': 'qnli', 
#     'set_of_data': 'validation', 
#     'globenc_config_name': 'Decomposition AbsDot Bias', 
#     'output_type': output_type
# }
# df1 = pd.read_pickle("/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[qnli]_[validation]_[output_qnli_bert-base-uncased_0001_SEED0042-checkpoint-16370]_[Decomposition No Bias].pkl")
# df2 = pd.read_pickle("/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[qnli]_[validation]_[output_qnli_bert-base-uncased_0001_SEED0042-checkpoint-16370]_[Decomposition AbsDot Bias].pkl")

i1 = {
    'masking_ratios': 0.7, 
    'model_checkpoint': '/home/modaresi/projects/globenc_analysis/outputs/models/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360', 
    'task_name': 'mnli', 
    'set_of_data': 'validation_matched', 
    'globenc_config_name': 'Decomposition No Bias', 
    'output_type': output_type
}
i2 = {
    'masking_ratios': 0.7, 
    'model_checkpoint': '/home/modaresi/projects/globenc_analysis/outputs/models/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360', 
    'task_name': 'mnli', 
    'set_of_data': 'validation_matched', 
    'globenc_config_name': 'Decomposition AbsDot Bias', 
    'output_type': output_type
}
df1 = pd.read_pickle("/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[mnli]_[validation_matched]_[output_mnli_bert-base-uncased_0001_SEED0042-checkpoint-61360]_[GlobEnc].pkl")
df2 = pd.read_pickle("/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[mnli]_[validation_matched]_[output_mnli_bert-base-uncased_0001_SEED0042-checkpoint-61360]_[Decomposition AbsDot Bias].pkl")


r1 = results.get(i1)
l1 = scipy.special.softmax(np.array(r1["logits"].tolist()), axis=-1)
# l1 = np.array(r1["logits"].tolist())

r2 = results.get(i2)
l2 = scipy.special.softmax(np.array(r2["logits"].tolist()), axis=-1)
# l2 = np.array(r2["logits"].tolist())
              
# np.argsort(np.abs(l1[:, 1] - l2[:, 0]))[-20:]
distances = []
for i in tqdm(range(len(df1))):
#     p = stats.spearmanr(df1.iloc[i]["importance_last_layer_aggregated"][:, 1], df2.iloc[i]["importance_last_layer_aggregated"][:, 1])[0]
#     p = stats.spearmanr(df1.iloc[i]["importance_last_layer_aggregated"][0, :], df2.iloc[i]["importance_last_layer_aggregated"][0, :])[0]
    p = np.max(df2.iloc[i]["importance_last_layer_aggregated"][2, :]) - np.min(df2.iloc[i]["importance_last_layer_aggregated"][2, :])
    distances.append(p)

In [ ]:
l1

In [ ]:
# for i in np.argsort(np.abs(l1[:, 2] - l2[:, 2]))[-50:]:
for i in np.argsort(distances)[-20:]:
    print(distances[i], l1[i], l2[i])
    if len(r1.iloc[i]["tokens"]) < 28 and np.argmax(r2.iloc[i]["logits"]) == r2.iloc[i]["label"]:
        print("###### ", i)
        print_preview(idx=i, discrete=False)
        print("No Bias")
        print(r1.iloc[i]["tokens"])
        print(r1.iloc[i]["logits"])
        print("AbsDot")
        print(r2.iloc[i]["tokens"])
        print(r2.iloc[i]["logits"])

In [ ]:
### No Bias
print(r1.iloc[3023]["tokens"])
print(r1.iloc[3023]["logits"])
### AbsDot
print(r2.iloc[3023]["tokens"])
print(r2.iloc[3023]["logits"])

# Final Figures

## 1) Main Figure

In [ ]:
def plot_main_fig(METRIC, filter_dataset, LEGEND_CONFIG_MAP, model="bert-base", save_fig=True):
    plt.figure(figsize=(6, 4))
    configs = [
        "Decomposition AbsDot Bias", "ALTI", 
        "IG_ALL_ZERO_NORM_prediction_based", "IXG_NORM_prediction_based", 
        "GlobEnc", "GlobEnc No Bias FFN",
        "Decomposition No Bias", "Decomposition AbsDot Bias No FFN", 
        "Decomposition AbsSim Bias", "Decomposition Norm Bias", "Decomposition Equal Bias",
        "+Decomposition AbsDot Bias", "Decomposition No Bias No FFN",
    ]
    markers = ['-o', '-D', '-s', '-^', '-*', '-+', '-x', '-.', '-X']
    cmap = plt.get_cmap("tab10")
    c = 0
    aggregated_results = {}
    for globenc_config_name in tqdm(configs):
        for output_type in results.indices['output_type']:
            for model_checkpoint in results.indices['model_checkpoint']:
                for task_name in results.indices['task_name']:
                    for set_of_data in results.indices['set_of_data']:
                        if filter_dataset and filter_dataset not in f"{task_name}_{globenc_config_name}_{output_type}" or model not in model_checkpoint:
                            continue
                        plot_y, plot_x = [], []
                        try:
                            for masking_ratio in sorted(results.indices['masking_ratios']):
                                if masking_ratio == 1.0:
                                    continue
                                index_to_value = {
                                    "masking_ratios": masking_ratio, 
                                    "model_checkpoint": model_checkpoint, 
                                    "task_name": task_name, 
                                    "set_of_data": set_of_data, 
                                    "globenc_config_name": globenc_config_name,
                                    "output_type": output_type,
                                }
                                metric_result = metric_function[METRIC](results, index_to_value)
                                plot_y.append(metric_result)
                                plot_x.append(masking_ratio)
                            label = f"{output_type} {globenc_config_name}"
                            
#                             print(label)
                            if label in LEGEND_CONFIG_MAP.keys():
                                label = LEGEND_CONFIG_MAP.get(label, label)
                                aggregated_results[label] = np.mean(plot_y[1:])
#                                 print(f"{task_name} {output_type} {globenc_config_name}".ljust(50), f"--> mean={np.mean(plot_y[1:]):.3f}|{np.mean(plot_y[1:])*100:.2f}, {len(plot_y[1:])}")
                                print(f"{METRIC}/{task_name}/{MASKING_STRATEGY}/{label}".ljust(30), f"--> mean={np.mean(plot_y[1:]):.3f}|{np.mean(plot_y[1:])*100:.2f}, {len(plot_y[1:])}")
#                                 print([f"{(y * 100):.2f}%" for y in plot_y])
                                lw = 3 if "DecompX" in label else 1
                                ms = 8 if "DecompX" in label else 6
                                if METRIC == "Accuracy":
                                    plot_y = [y * 100 for y in plot_y]
                                plt.plot([f"{int(x * 100)}%" for x in plot_x], plot_y, markers[c], color=cmap(c), label=label, lw=lw, markersize=ms)
                                c += 1
                            else:
                                pass
                        except Exception as e:
    #                         logginginging.warning(e)
                            pass
    plt.legend(facecolor='white', framealpha=0.8)
    plt.title(f"Mask K% of the {'Least' if MASKING_STRATEGY == 'min' else 'Most'} Important Tokens - {filter_dataset.upper()}", fontsize=13)
    plt.ylabel(f"{METRIC}")
    plt.xlabel("K%")
    if save_fig:
        plt.savefig(f"figs/main_{model}_{filter_dataset}_{MASKING_STRATEGY}_{METRIC}.pdf", bbox_inches='tight')
    plt.show()
    print(model_checkpoint)
    return aggregated_results

In [ ]:
LEGEND_CONFIG_MAP = {
    "classifier Decomposition AbsDot Bias": "DecompX",
#     "classifier +Decomposition AbsDot Bias": "+DecompX",
    "aggregated IG_ALL_ZERO_NORM_prediction_based": "Integrated Gradient",
    "aggregated IXG_NORM_prediction_based": "Gradient×Input",
    "aggregated GlobEnc": "GlobEnc",
#     "aggregated GlobEnc No Bias FFN": "GlobEnc FFN",
    "aggregated ALTI": "ALTI",
}
for filter_dataset in ['sst2', 'mnli', 'qnli', 'hatexplain']:  # 'sst2', 'mnli', 'qnli', 'hatexplain'
    for METRIC in ["AOPC", "Accuracy"]:  # "Accuracy", "AOPC+", "LOdds", "AOPC"
        a = plot_main_fig(METRIC, filter_dataset, LEGEND_CONFIG_MAP)

## 2) Ablation Analysis

In [ ]:
LEGEND_CONFIG_MAP = {
#     "aggregated GlobEnc": "GlobEnc",
    "classifier Decomposition AbsDot Bias": "DecompX",
    "aggregated Decomposition AbsDot Bias": "DecompX W/O Clf. Head",
    "classifier Decomposition No Bias": "DecompX W/O Bias",
    "classifier Decomposition AbsDot Bias No FFN": "DecompX W/O FFN",
#     "classifier Decomposition No Bias No FFN": "DecompX - FFN - Bias",
#     "aggregated Decomposition No Bias No FFN": "DecompX - FFN - Bias - Classification Head",
}
df = pd.DataFrame()
for filter_dataset in ['mnli', 'hatexplain', 'sst2', 'qnli', ]:  # 'sst2', 'mnli', 'qnli', 'hatexplain'
    for METRIC in ["AOPC"]:  # "Accuracy", "AOPC+", "LOdds", "AOPC"
        aggregated_results = plot_main_fig(METRIC, filter_dataset, LEGEND_CONFIG_MAP, save_fig=False)
        new_df = pd.DataFrame(aggregated_results.items(), columns=['Method', 'AOPC'])
        new_df["Dataset"] = filter_dataset.upper()
        print(new_df)
        df = df.append(new_df)

In [ ]:
patterns = ["...", "xxx", "++", '///', "|" , "-" , "+" , "//",  "O",  "*" ][:4]
l = ['DecompX W/O Clf. Head', 'DecompX W/O FFN', 'DecompX W/O Bias', 'DecompX', ]
dfp = df.pivot("Dataset", "Method", "AOPC")[l].loc[["QNLI", "SST2", "HATEXPLAIN", "MNLI"]]
ax = dfp.plot(
    kind='barh', figsize=(6, 4), width=0.85, 
#     color=plt.get_cmap("tab20b").colors[4:8][::-1], 
    color=[plt.get_cmap("summer")(c) for c in [0, 0.3, 0.5, 0.7]][::1], 
    edgecolor=(1,1,1, 0.5)
)
# plt.legend(loc='best')
# for c, container in enumerate(ax.containers):
#     bl = ax.bar_label(
#         container, 
#         labels=[l[c]] * len(l),
#         label_type="edge", color="black", padding=0, weight='normal'
#     )
#     for b in bl:
#         b.xy = (0.005, b.xy[1])
# ax.get_legend().remove()

### HATCH ###
# bars = ax.patches
# hatches = [h for h in patterns for _ in range(len(dfp))]
# print(hatches)
# for bar, hatch in zip(bars, hatches):
#     bar.set_hatch(hatch)
### HATCH ###

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title=None, loc='upper left', facecolor='white', framealpha=0.8)

plt.xlim(0, 0.73)
plt.xlabel("AOPC")
plt.tight_layout()
plt.savefig(f"figs/ablation_leaveOneOut_{MASKING_STRATEGY}_{METRIC}.pdf", bbox_inches='tight')
plt.show()

In [ ]:
LEGEND_CONFIG_MAP = {
    "aggregated GlobEnc": "GlobEnc",
#     "classifier Decomposition AbsDot Bias": "DecompX",
    "aggregated Decomposition AbsDot Bias": "DecompX W/O Clf. Head",
#     "classifier Decomposition No Bias": "DecompX W/O Bias",
#     "classifier Decomposition AbsDot Bias No FFN": "DecompX W/O FFN",
#     "classifier Decomposition No Bias No FFN": "DecompX W/O FFN/Bias",
    "aggregated Decomposition No Bias No FFN": "DecompX W/O FFN/Bias/Clf. Head",
    "aggregated GlobEnc No Bias FFN": "GlobEnc FFN",
}
df = pd.DataFrame()
for filter_dataset in ['sst2', 'mnli', 'qnli', 'hatexplain']:  # 'sst2', 'mnli', 'qnli', 'hatexplain'
    for METRIC in ["AOPC"]:  # "Accuracy", "AOPC+", "LOdds", "AOPC"
        aggregated_results = plot_main_fig(METRIC, filter_dataset, LEGEND_CONFIG_MAP, save_fig=False)
        new_df = pd.DataFrame(aggregated_results.items(), columns=['Method', 'AOPC'])
        new_df["Dataset"] = filter_dataset.upper()
        print(new_df)
        df = df.append(new_df)

In [ ]:
l = ['GlobEnc', "GlobEnc FFN", 'DecompX W/O FFN/Bias/Clf. Head', "DecompX W/O Clf. Head"]
dfp = df.pivot("Dataset", "Method", "AOPC")[l].loc[["SST2", "QNLI", "HATEXPLAIN", "MNLI"]]
ax = dfp.plot(
    kind='barh', figsize=(6, 4), width=0.85, 
#     color=plt.get_cmap("tab20b").colors[4:8][::-1], 
    color=[plt.get_cmap("plasma")(c) for c in [0.8, 0.9]] + [plt.get_cmap("summer")(c) for c in [0.6, 0.0]], 
    edgecolor=(1,1,1, 0.5)
)
# plt.legend(loc='best')
# for c, container in enumerate(ax.containers):
#     bl = ax.bar_label(
#         container, 
#         labels=[l[c]] * len(l),
#         label_type="edge", color="white", padding=0, weight='bold'
#     )
#     for b in bl:
#         b.xy = (0.005, b.xy[1])
# ax.get_legend().remove()

### HATCH ###
# bars = ax.patches
# hatches = [h for h in patterns for _ in range(len(dfp))]
# print(hatches)
# for bar, hatch in zip(bars, hatches):
#     bar.set_hatch(hatch)
### HATCH ###

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title=None, loc='upper left', facecolor='white', framealpha=0.8)
plt.xlabel("AOPC")
plt.xlim(0, 0.73)
plt.tight_layout()
plt.savefig(f"figs/ablation_decomp_{MASKING_STRATEGY}_{METRIC}.pdf", bbox_inches='tight')
plt.show()
dfp

In [ ]:
(red, *plt.get_cmap("tab10").colors[0:2])

In [ ]:
plt.get_cmap("tab10").colors[3]

## 3) Biases Analysis

In [ ]:
LEGEND_CONFIG_MAP = {
    "classifier Decomposition AbsDot Bias": "DecompX AbsDot Bias",
    "classifier Decomposition AbsSim Bias": "DecompX AbsSim Bias",
    "classifier Decomposition No Bias": "DecompX No Bias",
    "classifier Decomposition Equal Bias": "DecompX Equal Bias",
    "classifier Decomposition Norm Bias": "DecompX Norm Bias",
}
for filter_dataset in ['sst2', 'mnli', 'qnli', 'hatexplain']:  # 'sst2', 'mnli', 'qnli', 'hatexplain'
    for METRIC in ["Accuracy"]:  # "Accuracy", "AOPC+", "LOdds", "AOPC"
        plot_main_fig(METRIC, filter_dataset, LEGEND_CONFIG_MAP, save_fig=False)

## 4) Roberta

In [ ]:
LEGEND_CONFIG_MAP = {
    "classifier Decomposition AbsDot Bias": "DecompX",
#     "classifier +Decomposition AbsDot Bias": "+DecompX",
    "aggregated IG_ALL_ZERO_NORM_prediction_based": "Integrated Gradient",
    "aggregated IXG_NORM_prediction_based": "Gradient×Input",
    "aggregated GlobEnc": "GlobEnc",
    "aggregated GlobEnc No Bias FFN": "GlobEnc FFN",
    "aggregated ALTI": "ALTI",
}
for filter_dataset in ['mnli']:  # 'sst2', 'mnli', 'qnli', 'hatexplain'
    for METRIC in ["AOPC", "Accuracy"]:  # "Accuracy", "AOPC+", "LOdds", "AOPC"
        plot_main_fig(METRIC, filter_dataset, LEGEND_CONFIG_MAP, model="roberta", save_fig=True)

# Examples

In [ ]:
results.summary()

In [ ]:
globencs = dict()
for model_dataset_set in tqdm(MODEL_DATASET_SET, desc="Models_Dataset_Sets"):
    model_checkpoint, task_name, set_of_data = model_dataset_set
    for globenc_cfg_name in tqdm(GLOBENC_CONFIG_NAMES, desc="Globenc Configs"):
        file_name = f"[{task_name}]_[{set_of_data}]_[{'-'.join(model_checkpoint.split('/')[-2:])}]_[{globenc_cfg_name}]"
        
        df = pd.read_pickle(f"{OUTPUT_DIR}/{file_name}.pkl")
        globencs[f"{globenc_cfg_name}"] = df
        
        print(file_name)
        idx = 10
        print_globenc(
            df["globenc_last_layer"].iloc[idx], 
            df["tokens"].iloc[idx], 
            prefix=f"{globenc_cfg_name}:".ljust(13), 
            no_cls_sep=True,
            discrete=False
        )

In [ ]:
ratio = 0.8
index_to_value_d = {
    "masking_ratios": ratio, 
    "model_checkpoint": model_checkpoint, 
    "task_name": task_name, 
    "set_of_data": set_of_data, 
    "globenc_config_name": "decomp",
}

index_to_value_g = {
    "masking_ratios": ratio, 
    "model_checkpoint": model_checkpoint, 
    "task_name": task_name, 
    "set_of_data": set_of_data, 
    "globenc_config_name": "globenc",
}

df_d = results.get(index_to_value_d)
df_g = results.get(index_to_value_g)
idxs = np.linalg.norm(
    scipy.special.softmax(np.array(df_d["logits"].tolist()), axis=-1) - 
    scipy.special.softmax(np.array(df_g["logits"].tolist()), axis=-1)
    , axis=-1
).argsort()
# idxs = np.linalg.norm(np.array(df_d["logits"].tolist()) - np.array(df_g["logits"].tolist()), axis=-1).argsort()
idxs = np.flip(idxs)
idxs

In [ ]:
for idx in idxs[0:20]:
    print(idx)
    for cfg in ["decomp", "globenc", "globenc_ffn"]:
        print_globenc(
            globencs[cfg]["globenc_last_layer"].iloc[idx], 
            globencs[cfg]["tokens"].iloc[idx], 
            prefix=f"{cfg}:".ljust(13), 
            no_cls_sep=True,
            discrete=True
        )
    with pd.option_context('display.max_colwidth', None):
        print("decomp:")
        display(df_d.iloc[[idx]])
        print("globenc:")
        display(df_g.iloc[[idx]])

In [ ]:
import itertools
itertools.product(*results.indices.values)

In [ ]:
df = results.get({'masking_ratios': 0.0, 'model_checkpoint': '/home/modaresi/projects/globenc_analysis/outputs/models/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-10525', 'task_name': 'sst2', 'set_of_data': 'validation', 'globenc_config_name': 'decomp'})

In [ ]:
for i in sorted(results.indices['masking_ratios']):
    print(i)

# IG to Pandas Format

In [ ]:
# sal_path = "/home/modaresi/projects/globenc_analysis/outputs/integrated_saliencies/"
sal_path = "/home/modaresi/projects/globenc_analysis/outputs/saliencies/"

dataset_map = {
#     "cola": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[cola]_[validation]_[output_cola_bert-base-uncased_0001_SEED0042-checkpoint-1340]_[Decomposition LinearApproximation].pkl",
#     "qnli": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[qnli]_[validation]_[output_qnli_bert-base-uncased_0001_SEED0042-checkpoint-16370]_[Decomposition LinearApproximation].pkl",
#     "mnli": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[mnli]_[validation_matched]_[output_mnli_bert-base-uncased_0001_SEED0042-checkpoint-61360]_[Decomposition ZeroOriginApproximation].pkl",
#     "sst2": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[sst2]_[validation]_[output_sst2_bert-base-uncased_0001_SEED0042-checkpoint-10525]_[GlobEnc FFN LinearApproximation].pkl",
#     "hatexplain": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[hatexplain]_[validation]_[output_hatexplain_bert-base-uncased_0001_SEED0042-checkpoint-2405]_[GlobEnc].pkl",
    "mnli": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[mnli]_[validation_matched]_[WillHeld-roberta-base-mnli]_[Decomposition AbsDot Bias].pkl",
    "sst2": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[sst2]_[validation]_[WillHeld-roberta-base-sst2]_[Decomposition AbsDot Bias].pkl",
}

for file_name in tqdm(os.listdir(sal_path)):
    dataset_df = None
    for ds, path in dataset_map.items():
        if ds in file_name:
            dataset_df = pd.read_pickle(path)[["tokens", "logits", "label"]]
            lengths = dataset_df["tokens"].str.len()
    if dataset_df is None or not "roberta" in file_name:
        continue
    if ".npy" in file_name:
        a = np.load(f"{sal_path}{file_name}")
        importance = [np.expand_dims(a_i[:lengths[i]], axis=0) for i, a_i in enumerate(a)]
        df = pd.DataFrame({
            "importance_last_layer_aggregated": importance,
            "tokens": dataset_df["tokens"].values,
            "logits": dataset_df["logits"].values,
            "label": dataset_df["label"].values,
        })
        f = f"{sal_path}{file_name.split('.')[0]}.pkl"
        print(f)
        df.to_pickle(f)

# ALTI to Pandas Format

In [ ]:
# alti_path = "/home/modaresi/projects/globenc_analysis/notebooks/ALTI/transformer-contributions/data/validation/"
alti_path = "/home/modaresi/projects/globenc_analysis/temp/"
ALTI_OUTPUT_DIR = "/home/modaresi/projects/globenc_analysis/outputs/ALTI"

dataset_map = {
#     "cola": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[cola]_[validation]_[output_cola_bert-base-uncased_0001_SEED0042-checkpoint-1340]_[Decomposition AbsDot Bias].pkl",
#     "qnli": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[qnli]_[validation]_[output_qnli_bert-base-uncased_0001_SEED0042-checkpoint-16370]_[Decomposition AbsDot Bias].pkl",
#     "mnli": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[mnli]_[validation_matched]_[output_mnli_bert-base-uncased_0001_SEED0042-checkpoint-61360]_[Decomposition AbsDot Bias].pkl",
#     "sst2": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[sst2]_[validation]_[output_sst2_bert-base-uncased_0001_SEED0042-checkpoint-10525]_[Decomposition AbsDot Bias].pkl",
#     "hatexplain": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[hatexplain]_[validation]_[output_hatexplain_bert-base-uncased_0001_SEED0042-checkpoint-2405]_[Decomposition AbsDot Bias].pkl",
    "mnli": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[mnli]_[validation_matched]_[WillHeld-roberta-base-mnli]_[Decomposition AbsDot Bias].pkl",
    "sst2": "/home/modaresi/projects/globenc_analysis/outputs/globencs_v3/[sst2]_[validation]_[WillHeld-roberta-base-sst2]_[Decomposition AbsDot Bias].pkl",
}

for file_name in tqdm(os.listdir(alti_path)):
    for ds, path in dataset_map.items():
        if ds in file_name:
            dataset_df = pd.read_pickle(path)[["tokens", "logits", "label"]]
            lengths = dataset_df["tokens"].str.len()
    if ".npy" in file_name:
        a = np.load(f"{alti_path}{file_name}", allow_pickle=True).item()
        preds = a["pred_class"]
        a = a["ours"]
        importance = [np.expand_dims(a_i, axis=0) for i, a_i in enumerate(a)]
        df = pd.DataFrame({
            "importance_last_layer_aggregated": importance,
            "tokens": dataset_df["tokens"].values,
            "logits": dataset_df["logits"].values,
            "label": dataset_df["label"].values,
            "pred_class": preds,
        })
        fn = make_result_filename(dataset_df.attrs["task_name"], dataset_df.attrs["set_of_data"], dataset_df.attrs["model_checkpoint"], "ALTI")
        f = f"{ALTI_OUTPUT_DIR}/{fn}.pkl"
        print(f)
        df.to_pickle(f)
df

In [ ]:
dataset_df.attrs

In [ ]:
idx = 50
print(len(df.iloc[idx]["tokens"]))
print(len(df.iloc[idx]["importance_last_layer_aggregated"][0]))

In [ ]:
a = np.load(f"/home/modaresi/projects/globenc_analysis/outputs/integrated_saliencies/[qnli]_[validation]_[output_qnli_bert-base-uncased_0001_SEED0042-checkpoint-16370]_[IG_ALL_PAD_SUM_label_based].npy")
b = np.load(f"/home/modaresi/projects/globenc_analysis/outputs/integrated_saliencies/[qnli]_[validation]_[output_qnli_bert-base-uncased_0001_SEED0042-checkpoint-16370]_[ALL_PAD]_[SUM].npy")

In [ ]:
a = np.load(f"{path}bert_sst2_attributions.npy", allow_pickle=True)

# Create DecompX+

In [ ]:
for dir_model_dataset_set in tqdm(DIR_MODEL_DATASET_SET, desc="Models_Dataset_Sets"):
    dir, model_checkpoint, task_name, set_of_data = dir_model_dataset_set
    for cfg_name in tqdm(GLOBENC_CONFIG_NAMES, desc="Configs"):
        file_name = make_result_filename(task_name, set_of_data, model_checkpoint, cfg_name)
        try:
            df = pd.read_pickle(f"{dir}/{file_name}.pkl")
            df["importance_last_layer_classifier"] = df["importance_last_layer_classifier"].apply(lambda x: np.abs(x))
            file_name = make_result_filename(task_name, set_of_data, model_checkpoint, f"+{cfg_name}")
            print(file_name)
            df.to_pickle(f"{dir}/{file_name}.pkl")
        except Exception as e:
            print(e)
            continue

In [ ]:
df["importance_last_layer_classifier"] = df["importance_last_layer_classifier"].apply(lambda x: np.abs(x))

In [ ]:
df["importance_last_layer_classifier"]